# Model Training

This notebook handles the training of all models with performance monitoring:
1. Original models (1DCNN, RNN, DNN, LSTM, BiLSTM)
2. Hybrid models (CNN-LSTM, CNN-BiLSTM)
3. Memory and inference time tracking
4. Training metrics logging

In [ ]:
import os
import time
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import psutil

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)

## 1. Load Data and Model Definitions

In [ ]:
# Load preprocessed data
X_train = np.load('preprocessed/X_train.npy')
X_test = np.load('preprocessed/X_test.npy')
y_train = np.load('preprocessed/y_train.npy')
y_test = np.load('preprocessed/y_test.npy')

# Reshape data for deep learning models
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Load model functions
with open('models/model_functions.pkl', 'rb') as f:
    model_functions = pickle.load(f)

print("Data shapes:")
print(f"X_train: {X_train_reshaped.shape}")
print(f"X_test: {X_test_reshaped.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

## 2. Performance Monitoring Functions

In [ ]:
def measure_inference_time(model, X_test, batch_size=32, num_runs=100):
    """Measure average inference time"""
    times = []
    for _ in range(num_runs):
        start_time = time.time()
        model.predict(X_test[:batch_size], verbose=0)
        times.append(time.time() - start_time)
    
    avg_time = np.mean(times) * 1000  # Convert to milliseconds
    std_time = np.std(times) * 1000
    return avg_time, std_time

def get_model_memory_usage(model):
    """Get model memory usage in MB"""
    process = psutil.Process(os.getpid())
    baseline = process.memory_info().rss / 1024 / 1024
    
    # Force some predictions to load model into memory
    model.predict(X_test_reshaped[:1], verbose=0)
    
    after_load = process.memory_info().rss / 1024 / 1024
    return after_load - baseline

def log_training_metrics(history, model_name):
    """Log training metrics to file"""
    with open(f'logs/{model_name}_training.log', 'w') as f:
        f.write(f"Training metrics for {model_name}\n")
        f.write("-" * 50 + "\n")
        for metric, values in history.history.items():
            f.write(f"\n{metric}:\n")
            for epoch, value in enumerate(values):
                f.write(f"Epoch {epoch+1}: {value:.4f}\n")

## 3. Model Training Loop

In [ ]:
results = {}
BATCH_SIZE = 128
EPOCHS = 50

# Clear any existing models
tf.keras.backend.clear_session()

for name, create_model in model_functions.items():
    print(f"\nTraining {name}...")
    print("-" * 50)
    
    # Create model
    model = create_model()
    
    # Log initial memory usage
    initial_memory = get_model_memory_usage(model)
    print(f"Initial memory usage: {initial_memory:.2f} MB")
    
    # Compile model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=0.0001
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir=f'logs/{name}',
            histogram_freq=1
        )
    ]
    
    # Train model
    history = model.fit(
        X_train_reshaped,
        y_train,
        validation_split=0.2,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    
    # Measure final memory usage
    final_memory = get_model_memory_usage(model)
    print(f"Final memory usage: {final_memory:.2f} MB")
    
    # Measure inference time
    avg_time, std_time = measure_inference_time(model, X_test_reshaped)
    print(f"Average inference time: {avg_time:.2f} ± {std_time:.2f} ms")
    
    # Evaluate model
    y_pred = np.argmax(model.predict(X_test_reshaped, batch_size=BATCH_SIZE), axis=1)
    
    # Store results
    results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'history': history.history,
        'model': model,
        'predictions': y_pred,
        'memory_usage': final_memory,
        'inference_time': avg_time,
        'inference_std': std_time
    }
    
    # Print results
    print(f"\n{name} Results:")
    print(f"Accuracy: {results[name]['accuracy']:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    # Log training metrics
    log_training_metrics(history, name)
    
    # Save model
    model.save(f'models/{name}.h5')
    
    # Clear session for next model
    tf.keras.backend.clear_session()

## 4. Compare Model Metrics

In [ ]:
# Create comparison DataFrame
comparison_data = [
    {
        'Model': name,
        'Accuracy': res['accuracy'],
        'Memory (MB)': res['memory_usage'],
        'Inference (ms)': res['inference_time'],
        'Parameters': res['model'].count_params()
    }
    for name, res in results.items()
]

import pandas as pd
df_comparison = pd.DataFrame(comparison_data)
print("Model Comparison:")
print(df_comparison.to_string(index=False))

# Plot comparisons
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Accuracy comparison
ax1.bar(df_comparison['Model'], df_comparison['Accuracy'])
ax1.set_title('Model Accuracy')
ax1.set_xticklabels(df_comparison['Model'], rotation=45)
ax1.set_ylabel('Accuracy')

# Memory usage comparison
ax2.bar(df_comparison['Model'], df_comparison['Memory (MB)'])
ax2.set_title('Memory Usage')
ax2.set_xticklabels(df_comparison['Model'], rotation=45)
ax2.set_ylabel('Memory (MB)')

# Inference time comparison
ax3.bar(df_comparison['Model'], df_comparison['Inference (ms)'])
ax3.set_title('Inference Time')
ax3.set_xticklabels(df_comparison['Model'], rotation=45)
ax3.set_ylabel('Time (ms)')

# Parameter count comparison
ax4.bar(df_comparison['Model'], df_comparison['Parameters'])
ax4.set_title('Model Parameters')
ax4.set_xticklabels(df_comparison['Model'], rotation=45)
ax4.set_ylabel('Number of Parameters')

plt.tight_layout()
plt.show()

# Save results
with open('results/training_results.pkl', 'wb') as f:
    pickle.dump(results, f)

df_comparison.to_csv('results/model_comparison.csv', index=False)

print("\nResults saved successfully!")